In [86]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [87]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196'

In [88]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [89]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,jQTE9N9ZKbguudoFXjK93siPMsR0RxJ9iWe2ji9sbRkVxy86,Wpw-SB6IFCCehkR02j8k1fbAO7Pn6f9dxlW-RjICs-Xl_o...,OnW1Dw_JX8vapRJEb-ZVjqdKVknpD2IGd6ZWowLTrkmoEe...,kevyzhou,7,1656398187000,1,jQTE9N9ZKbguudoFXjK93siPMsR0RxJ9iWe2ji9sbRkVxy86,kevyzhou,650,I,246,233,False,False,True,False
1,m-qUxgZJLteV8W5Oqwwn50XfK0wXDtalCaVm9reptusp2JY,HG1ubDWkjeBciVDvB0bzOVDPf1SNPNt6Pk23ctu6fyQihA,WhTl_YwHuw06oZYoBvabD1I06ZBEUcelhXUlBDNnPXSKn-...,milala,3838,1656400235000,403,m-qUxgZJLteV8W5Oqwwn50XfK0wXDtalCaVm9reptusp2JY,milala,566,I,159,128,False,False,True,False
2,ZjryB2uaOap3yknlYg2NAM0KT0lJ2kZGxnji3SG0hwrSBgo,MrWHYI6liJ4DJ07BK8x2oTa9cI6yNbE_9tj1PVwrUVoLoc8,JfpGAp6IYvz9ExTxKZ-R0T-oOATXhtI1KRRN5Y4nb6GkOm...,Liquid Kurumx,3440,1656394547000,114,ZjryB2uaOap3yknlYg2NAM0KT0lJ2kZGxnji3SG0hwrSBgo,Liquid Kurumx,1006,I,97,40,False,False,False,True
3,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,K4FBeziXvtE_px9iFYbp72rXuEhJO_W21lbvRF5VgM4Spw,502mQ2ZxwRwZDIiwzDF_nEvUBsq8qhU9kG_WayPz4qz9DE...,BC Socks,3552,1656399877000,74,mn3OHPDTDc5CsloPPGlHOQOq0Vqagk66ReC61-8SuXUxqH0,BC Socks,1210,I,147,71,True,False,False,False
4,MPDXCwKEH-yDsQmyVYAGl5lzhjZCjAOGp8CSliPtRmWIT9E,B26caZessjQg8cDRq7iPTNJsoK-gn6J4Yc4nRXltcIAEVQ,8T0ADJVvo3K_K4O--Eq-rTIyQ0Pnb_z-EYjDf2QTy3C95O...,TSM FTX Kiyoon,21,1656302298000,148,MPDXCwKEH-yDsQmyVYAGl5lzhjZCjAOGp8CSliPtRmWIT9E,TSM FTX Kiyoon,835,I,136,87,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,v0V8za9lUe9Sal3pWMudPbeLKAV9KJam1IhcUktvoCpwFQ,FjLKIVYYUSrV37F_ZcBfkdPfRtkqXlYgw5u8WWv6WI7MYX...,TyIer Franklin,585,1656399634000,37,nRppazjRzzu6x6SVAZZw__qqNZzoL8jUmz6oA5PcLCFuYis,TyIer Franklin,693,I,94,50,False,False,True,True
81,rseOMGMOy6sMmC_BEUN8Rrzur0bedb1TyOJbKvdxThJtAhE,W3cdKoMwYdbDhIFn_7ZYvdTZmXpcdO6UzedkNu_TDBppjPY,lLtsw7rM-yW8RierLNxTKjpGlfn01KJr4Jdovj-7jqNenj...,Ramblinnn,3591,1656311786000,12,rseOMGMOy6sMmC_BEUN8Rrzur0bedb1TyOJbKvdxThJtAhE,Ramblinnn,731,I,97,54,False,False,False,False
82,qIxHRxoAzqUb-gy4NW_VQd_BpacB48qjlem7BGNCXz8Cmj0,k4yVx5UMYLD2EkUnLkCahkVvybxAKTni9NX2n7P2n4ddSg,4OhcQyoMGiGG4HQWXHn15EYNSojNWYHRx3upp3L_xGniI4...,JimLoveAngie,4568,1656387294000,140,qIxHRxoAzqUb-gy4NW_VQd_BpacB48qjlem7BGNCXz8Cmj0,JimLoveAngie,652,I,109,73,False,False,True,False
83,NI1BS8WAKRRk2qPubjPi3XGdv9xaKjPzJQuAzpie33cm-Fxq,qgL9DKswEAkNWKYHk6B4AO2-tlw4a3PfJdf0t0FLPjKT5n...,P6tuguVlHjl_9KVogt65c9N9Q1Hq34UMLAE2pqnweLIQKM...,OatmealWater,5271,1656387693000,48,NI1BS8WAKRRk2qPubjPi3XGdv9xaKjPzJQuAzpie33cm-Fxq,OatmealWater,560,I,160,113,False,False,True,False


In [90]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [91]:
len(uniq_matches_id)

2728

In [92]:
len(matches_asset)

3174

In [93]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [94]:
len(uniq_matches)

2728

In [95]:
matches_asset[0]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [96]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [97]:
len(latest_matches)

825

In [98]:
# latest_matches[0]['info']

In [99]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for index, augment in enumerate(participant['augments']):
            match[f'augment{index}'] = augment

        for index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{index}'] = item

        matches_array.append(match)

In [100]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4356726048',
 'placement': 6,
 'augment0': 'TFT7_Augment_AxiomArc1',
 'augment1': 'TFT6_Augment_SecondWind1',
 'augment2': 'TFT7_Augment_DragonmancerInspire',
 'Set7_Cavalier': 0,
 'Set7_Dragonmancer': 2,
 'Set7_Jade': 1,
 'Set7_Legend': 0,
 'Set7_Ragewing': 1,
 'Set7_Shapeshifter': 1,
 'Set7_Shimmerscale': 0,
 'Set7_Swiftshot': 0,
 'Set7_Tempest': 0,
 'TFT7_Sett': 2,
 'TFT7_Karma': 2,
 'TFT7_Ashe': 2,
 'TFT7_LeeSin': 2,
 'TFT7_LeeSin_item0': 'TFT_Item_RedBuff',
 'TFT7_LeeSin_item1': 'TFT_Item_BrambleVest',
 'TFT7_Swain': 2,
 'TFT7_Swain_item0': 'TFT_Item_StatikkShiv',
 'TFT7_Swain_item1': 'TFT_Item_Quicksilver',
 'TFT7_Swain_item2': 'TFT_Item_ArchangelsStaff',
 'TFT7_Volibear': 2,
 'TFT7_Hecarim': 1,
 'TFT7_Neeko': 2,
 'TFT7_Neeko_item0': 'TFT_Item_DragonsClaw',
 'TFT7_Neeko_item1': 'TFT_Item_FrozenHeart',
 'TFT7_Neeko_item2': 'TFT_Item_Bloodthirster'}

In [101]:
matches_league_df = pd.json_normalize(matches_array)

In [102]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Cavalier,Set7_Dragonmancer,Set7_Jade,Set7_Legend,Set7_Ragewing,...,TFT7_TahmKench_item2,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Aatrox_item2,TFT7_Skarner_item2
0,NA1_4356726048,6,TFT7_Augment_AxiomArc1,TFT6_Augment_SecondWind1,TFT7_Augment_DragonmancerInspire,0.0,2.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4356726048,7,TFT6_Augment_SecondWind1,TFT6_Augment_CelestialBlessing1,TFT6_Augment_FuturePeepers,0.0,0.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4356726048,8,TFT6_Augment_TinyTitans,TFT6_Augment_MakeshiftArmor1,TFT6_Augment_Diversify2,0.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4356726048,5,TFT6_Augment_SecondWind1,TFT6_Augment_Weakspot,TFT6_Augment_MakeshiftArmor2,NaN,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4356726048,1,TFT6_Augment_Electrocharge1,TFT6_Augment_Diversify1,TFT7_Augment_GuildEmblem,2.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,NA1_4356236024,8,TFT7_Augment_CannoneerRicochet,TFT6_Augment_SecondWind2,TFT7_Augment_ThinkFast,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6596,NA1_4356236024,5,TFT7_Augment_AssassinEmblem,TFT7_Augment_GadgetExpert,TFT6_Augment_CelestialBlessing3,NaN,0.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6597,NA1_4356236024,4,TFT7_Augment_Bloodlust1,TFT6_Augment_Ascension,TFT7_Augment_UrfsGrabBag2,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TFT_Item_ArchangelsStaff,NaN
6598,NA1_4356236024,1,TFT6_Augment_CyberneticImplants2,TFT6_Augment_PortableForge,TFT6_Augment_ForceOfNature,1.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Cavalier,Set7_Dragonmancer,Set7_Jade,Set7_Legend,Set7_Ragewing,...,TFT7_TahmKench_item2,TFT7_Vladimir_item0,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_AurelionSol,TFT7_AurelionSol_item0,TFT7_AurelionSol_item1,TFT7_AurelionSol_item2,TFT7_Aatrox_item2,TFT7_Skarner_item2
0,NA1_4356726048,6,TFT7_Augment_AxiomArc1,TFT6_Augment_SecondWind1,TFT7_Augment_DragonmancerInspire,0.0,2.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4356726048,7,TFT6_Augment_SecondWind1,TFT6_Augment_CelestialBlessing1,TFT6_Augment_FuturePeepers,0.0,0.0,2.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4356726048,8,TFT6_Augment_TinyTitans,TFT6_Augment_MakeshiftArmor1,TFT6_Augment_Diversify2,0.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4356726048,5,TFT6_Augment_SecondWind1,TFT6_Augment_Weakspot,TFT6_Augment_MakeshiftArmor2,NaN,2.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4356726048,1,TFT6_Augment_Electrocharge1,TFT6_Augment_Diversify1,TFT7_Augment_GuildEmblem,2.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,NA1_4356236024,8,TFT7_Augment_CannoneerRicochet,TFT6_Augment_SecondWind2,TFT7_Augment_ThinkFast,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6596,NA1_4356236024,5,TFT7_Augment_AssassinEmblem,TFT7_Augment_GadgetExpert,TFT6_Augment_CelestialBlessing3,NaN,0.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6597,NA1_4356236024,4,TFT7_Augment_Bloodlust1,TFT6_Augment_Ascension,TFT7_Augment_UrfsGrabBag2,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TFT_Item_ArchangelsStaff,NaN
6598,NA1_4356236024,1,TFT6_Augment_CyberneticImplants2,TFT6_Augment_PortableForge,TFT6_Augment_ForceOfNature,1.0,0.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_matches.pickle'))